In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
data=[(1,	1),
(2,	1),
(3,	1),
(4,	2),
(5,	1),
(6,	2),
(7,	2)]
columns =["id","num"]
df1= spark.createDataFrame(data,columns)
df1.show()

+---+---+
| id|num|
+---+---+
|  1|  1|
|  2|  1|
|  3|  1|
|  4|  2|
|  5|  1|
|  6|  2|
|  7|  2|
+---+---+



In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *


employee_data = [
    (1, "Alice", "HR", 5000,"hyd"),
    (2, "Bob", "IT", 600,"bang"),
    (3, "Charlie", "Finance", 7000,"mumbai"),
    (4, "David", "IT", 5500,"bang"),
    (5, "Eve", "HR", 4800,"hyd")
]
employee_schema = StructType([StructField("id",IntegerType(),False),
                             StructField("name",StringType(),True),
                             StructField("dept",StringType(),True),
                             StructField("salary",IntegerType(),True),
                             StructField("city",StringType(),True)])
emp_df = spark.createDataFrame(employee_data,employee_schema)
emp_df.show()

+---+-------+-------+------+------+
| id|   name|   dept|salary|  city|
+---+-------+-------+------+------+
|  1|  Alice|     HR|  5000|   hyd|
|  2|    Bob|     IT|   600|  bang|
|  3|Charlie|Finance|  7000|mumbai|
|  4|  David|     IT|  5500|  bang|
|  5|    Eve|     HR|  4800|   hyd|
+---+-------+-------+------+------+



In [0]:
#selecting the columns in different ways
#select specific columns, and only those coluns appar
empdf_selects_all = emp_df.select("*")
empdf_name = emp_df.select("name")
empdf_id = emp_df.select(col("id"))
empdf_slary=emp_df.select(emp_df["salary"])
empdf_dept =emp_df.select(emp_df.dept)
empdf_fewcol =emp_df.select("name",col("id"),emp_df["salary"],emp_df.dept)
empdf_selects_all.show()
empdf_name.show()
empdf_id.show()
empdf_slary.show()
empdf_dept.show()
empdf_fewcol.show()

+---+-------+-------+------+------+
| id|   name|   dept|salary|  city|
+---+-------+-------+------+------+
|  1|  Alice|     HR|  5000|   hyd|
|  2|    Bob|     IT|   600|  bang|
|  3|Charlie|Finance|  7000|mumbai|
|  4|  David|     IT|  5500|  bang|
|  5|    Eve|     HR|  4800|   hyd|
+---+-------+-------+------+------+

+-------+
|   name|
+-------+
|  Alice|
|    Bob|
|Charlie|
|  David|
|    Eve|
+-------+

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
+---+

+------+
|salary|
+------+
|  5000|
|   600|
|  7000|
|  5500|
|  4800|
+------+

+-------+
|   dept|
+-------+
|     HR|
|     IT|
|Finance|
|     IT|
|     HR|
+-------+

+-------+---+------+-------+
|   name| id|salary|   dept|
+-------+---+------+-------+
|  Alice|  1|  5000|     HR|
|    Bob|  2|   600|     IT|
|Charlie|  3|  7000|Finance|
|  David|  4|  5500|     IT|
|    Eve|  5|  4800|     HR|
+-------+---+------+-------+



In [0]:
#Expressions are used to perform computations on DataFrame columns
#COLUMN EXPRESSIONS
empdf_slary_new1=emp_df.select(col("salary")+1000)
empdf_slary_new1.show()
#adding new column, updating old column with "withColumn"
#syntax - df.withColumn(newcolumnname or exisitingcolumnname, expression )
empdf_newdf = emp_df.withColumn("newSlaary",col("salary")+1000)
empdf_newdf.show()
empdf_updateddf = emp_df.withColumn("salary",col("salary")+1000)
empdf_updateddf.show()
#LITERAL EXPRESSSION
empdf_lit = emp_df.select("*",lit("y").alias("status")) #selects everything and adds new column
empdf_lit.show()
#USER DEFINED EXPRESSSION
def upper_func(fname):
    return (str.upper(fname))

upper_name_udf = udf(upper_func,StringType())

empdf_name_upper = emp_df.withColumn("Upper_name",upper_name_udf(col("name")))
empdf_name_upper.show()

+---------------+
|(salary + 1000)|
+---------------+
|           6000|
|           1600|
|           8000|
|           6500|
|           5800|
+---------------+

+---+-------+-------+------+------+---------+
| id|   name|   dept|salary|  city|newSlaary|
+---+-------+-------+------+------+---------+
|  1|  Alice|     HR|  5000|   hyd|     6000|
|  2|    Bob|     IT|   600|  bang|     1600|
|  3|Charlie|Finance|  7000|mumbai|     8000|
|  4|  David|     IT|  5500|  bang|     6500|
|  5|    Eve|     HR|  4800|   hyd|     5800|
+---+-------+-------+------+------+---------+

+---+-------+-------+------+------+
| id|   name|   dept|salary|  city|
+---+-------+-------+------+------+
|  1|  Alice|     HR|  6000|   hyd|
|  2|    Bob|     IT|  1600|  bang|
|  3|Charlie|Finance|  8000|mumbai|
|  4|  David|     IT|  6500|  bang|
|  5|    Eve|     HR|  5800|   hyd|
+---+-------+-------+------+------+

+---+-------+-------+------+------+------+
| id|   name|   dept|salary|  city|status|
+---+------

In [0]:
#ALIAS
emp_df_alias = emp_df.select(col("id").alias("empid")) #using select
emp_df_alias.show()
emp_df_alias_with_column = emp_df.withColumn("emp_id",col("id")) #using select
emp_df_alias_with_column.show()
#filter or Where
#syntax - df.filter(column expression)
#syntax - df.where(column expression)
emp_df_filter = emp_df.filter(col("salary")>5000) #using select
emp_df_filter.show()
emp_df_where = emp_df.where(col("salary")>5000) #using select
emp_df_where.show()
#literal
empdf_lit = emp_df.select("*",lit("y").alias("status")) #selects everything and adds  
                                                        #new column
empdf_lit.show()
empdf_lit_withcol = emp_df.withColumn("status",lit("y"))
empdf_lit_withcol.show()
#adding columns
#use withColumn to add new column oe update existing columns
#renaming columns 
#syntax  - df.withColumnRenamed(old column name, newcolumn name)
emp_df_id_renamed = emp_df.withColumnRenamed("id","emp_idd")
emp_df_id_renamed.show()
#removing columns
#syntax  - df.drop(column names)
emp_drop1column = emp_df.drop("id")
emp_drop1column.show()
emp_dropmulticolumn = emp_df.drop("name","salary")
emp_dropmulticolumn.show()
#cast columns
emp_df_cst= emp_df.select(col('id').cast("string"))
emp_df_cst.printSchema()
emp_df_cst_withcol= emp_df.withColumn("empidddddd",col('id').cast("string"))
emp_df_cst_withcol.printSchema()




+-----+
|empid|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
+-----+

+---+-------+-------+------+------+------+
| id|   name|   dept|salary|  city|emp_id|
+---+-------+-------+------+------+------+
|  1|  Alice|     HR|  5000|   hyd|     1|
|  2|    Bob|     IT|   600|  bang|     2|
|  3|Charlie|Finance|  7000|mumbai|     3|
|  4|  David|     IT|  5500|  bang|     4|
|  5|    Eve|     HR|  4800|   hyd|     5|
+---+-------+-------+------+------+------+

+---+-------+-------+------+------+
| id|   name|   dept|salary|  city|
+---+-------+-------+------+------+
|  3|Charlie|Finance|  7000|mumbai|
|  4|  David|     IT|  5500|  bang|
+---+-------+-------+------+------+

+---+-------+-------+------+------+
| id|   name|   dept|salary|  city|
+---+-------+-------+------+------+
|  3|Charlie|Finance|  7000|mumbai|
|  4|  David|     IT|  5500|  bang|
+---+-------+-------+------+------+

+---+-------+-------+------+------+------+
| id|   name|   dept|salary|  city|status|
+---+-------+------

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
#UNION and UNION ALL is same for dataframe
#pyspark sql it is different 
student_data=[(10 ,'Anil',50000, 49),
(11 ,'Vikas',75000,  20),
(12 ,'Nisha',40000,  45),
(13 ,'Nidhi',60000,  50),
(14 ,'Priya',80000,  18),
(15 ,'Mohit',45000,  35),
(16 ,'Rajesh',90000, 43),
(17 ,'Raman',55000, 39),
(18 ,'Sam',65000,   33)]
student_data2=[(100 ,'Anila',50000, 49),
(101 ,'Viky',75000,  20),
(102 ,'Nishy',40000,  45),
(103 ,'nick',60000,  50),
(104 ,'Pritham',80000,  18)]
student_data_duplicate=[(18 ,'Sam',65000,   33),
(18 ,'Sam',55000,   33),
(23,"rai",50000,46)]
student_data_morecolms=[(10 ,'Anil',50000, 49,"A"),
(101 ,'Viky',75000,  20,"B")]
student_data_jumbledcolms=[(10 ,50000,'Anil', 49),
(101 ,75000, 'Viky', 20)]
student_schema = StructType([StructField("id",IntegerType(),False),
                             StructField("name",StringType(),True),
                             StructField("salary",IntegerType(),True),
                             StructField("marks",IntegerType(),True)])
student_schema_more_col = StructType([StructField("id",IntegerType(),False),
                             StructField("name",StringType(),True),
                             StructField("salary",IntegerType(),True),
                             StructField("marks",IntegerType(),True),
                             StructField("grade",StringType(),True)])
student_schema_jumbled_col = StructType([StructField("id",IntegerType(),False),
                            StructField("salary",IntegerType(),True),
                             StructField("name",StringType(),True),
                             StructField("marks",IntegerType(),True)])

mystudentdf1= spark.createDataFrame(student_data,student_schema)
mystudentdf1.show()
mystudentdf2= spark.createDataFrame(student_data2,student_schema)
mystudentdf2.show()
mystudentdfduplicate= spark.createDataFrame(student_data_duplicate,student_schema)
mystudentdfduplicate.show()
mystudentdfmorecol= spark.createDataFrame(student_data_morecolms,student_schema_more_col)
mystudentdfmorecol.show()
mystudentdfjumbledcolms= spark.createDataFrame(student_data_jumbledcolms,student_schema_jumbled_col)
mystudentdfjumbledcolms.show()
mystudentdf1.union(mystudentdf2).show()
mystudentdf1.unionAll(mystudentdf2).show()
mystudentdf1.union(mystudentdfduplicate).show()
mystudentdf1.unionAll(mystudentdfduplicate).show()
mystudentdf1.union(mystudentdfjumbledcolms).show()
mystudentdf1.unionAll(mystudentdfjumbledcolms).show()
mystudentdf1.unionByName(mystudentdfjumbledcolms).show()
#mystudentdf1.union(mystudentdfmorecol).show() #error
#mystudentdf1.unionAll(mystudentdfmorecol).show()#error





+---+------+------+-----+
| id|  name|salary|marks|
+---+------+------+-----+
| 10|  Anil| 50000|   49|
| 11| Vikas| 75000|   20|
| 12| Nisha| 40000|   45|
| 13| Nidhi| 60000|   50|
| 14| Priya| 80000|   18|
| 15| Mohit| 45000|   35|
| 16|Rajesh| 90000|   43|
| 17| Raman| 55000|   39|
| 18|   Sam| 65000|   33|
+---+------+------+-----+

+---+-------+------+-----+
| id|   name|salary|marks|
+---+-------+------+-----+
|100|  Anila| 50000|   49|
|101|   Viky| 75000|   20|
|102|  Nishy| 40000|   45|
|103|   nick| 60000|   50|
|104|Pritham| 80000|   18|
+---+-------+------+-----+

+---+----+------+-----+
| id|name|salary|marks|
+---+----+------+-----+
| 18| Sam| 65000|   33|
| 18| Sam| 55000|   33|
| 23| rai| 50000|   46|
+---+----+------+-----+

+---+----+------+-----+-----+
| id|name|salary|marks|grade|
+---+----+------+-----+-----+
| 10|Anil| 50000|   49|    A|
|101|Viky| 75000|   20|    B|
+---+----+------+-----+-----+

+---+------+----+-----+
| id|salary|name|marks|
+---+------+----+--

In [0]:

mystudentdf1.createOrReplaceTempView("mystudentdf1_tbl")
mystudentdf2.createOrReplaceTempView("mystudentdf2_tbl")
mystudentdfduplicate.createOrReplaceTempView("mystudentdfduplicate_tbl")
mystudentdfjumbledcolms.createOrReplaceTempView("mystudentdfjumbledcolms_tbl")
mystudentdfjumbledcolms.createOrReplaceTempView("mystudentdfjumbledcolms_tbl")
mystudentdfmorecol.createOrReplaceTempView("mystudentdfmorecol_tbl")
spark.sql("SELECT * FROM mystudentdf1_tbl union SELECT * FROM mystudentdf2_tbl").show()
spark.sql("SELECT * FROM mystudentdf1_tbl union all SELECT * FROM mystudentdf2_tbl").show()
spark.sql("SELECT * FROM mystudentdf1_tbl union SELECT * FROM mystudentdfduplicate_tbl").show()
spark.sql("SELECT * FROM mystudentdf1_tbl union all SELECT * FROM mystudentdfduplicate_tbl").show()
spark.sql("SELECT * FROM mystudentdf1_tbl union SELECT * FROM mystudentdfjumbledcolms_tbl").show()
spark.sql("SELECT * FROM mystudentdf1_tbl union all SELECT * FROM mystudentdfjumbledcolms_tbl").show()
spark.sql("SELECT * FROM mystudentdf1_tbl union SELECT * FROM mystudentdfmorecol_tbl").show() #error
spark.sql("SELECT * FROM mystudentdf1_tbl union all SELECT * FROM mystudentdfmorecol_tbl").show() #error

+---+-------+------+-----+
| id|   name|salary|marks|
+---+-------+------+-----+
| 10|   Anil| 50000|   49|
| 11|  Vikas| 75000|   20|
| 12|  Nisha| 40000|   45|
| 13|  Nidhi| 60000|   50|
| 14|  Priya| 80000|   18|
| 15|  Mohit| 45000|   35|
| 16| Rajesh| 90000|   43|
| 17|  Raman| 55000|   39|
| 18|    Sam| 65000|   33|
|100|  Anila| 50000|   49|
|101|   Viky| 75000|   20|
|102|  Nishy| 40000|   45|
|103|   nick| 60000|   50|
|104|Pritham| 80000|   18|
+---+-------+------+-----+

+---+-------+------+-----+
| id|   name|salary|marks|
+---+-------+------+-----+
| 10|   Anil| 50000|   49|
| 11|  Vikas| 75000|   20|
| 12|  Nisha| 40000|   45|
| 13|  Nidhi| 60000|   50|
| 14|  Priya| 80000|   18|
| 15|  Mohit| 45000|   35|
| 16| Rajesh| 90000|   43|
| 17|  Raman| 55000|   39|
| 18|    Sam| 65000|   33|
|100|  Anila| 50000|   49|
|101|   Viky| 75000|   20|
|102|  Nishy| 40000|   45|
|103|   nick| 60000|   50|
|104|Pritham| 80000|   18|
+---+-------+------+-----+

+---+------+------+-----+


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-2433231397931095>", line 13, in <module>
    spark.sql("SELECT * FROM mystudentdf1_tbl union SELECT * FROM mystudentdfmorecol_tbl").show()
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 48, in wrapper
    res = func(*args, **kwargs)
  File "/databricks/spark/python/pyspark/sql/session.py", line 1387, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
  File "/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/databricks/spark/python/pyspark/errors/exceptions.py", line 234, in deco
    raise converted from None
pyspark.errors.exceptions.AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same n

In [0]:
spark.sql("SELECT * FROM mystudentdf1_tbl union SELECT * FROM mystudentdf2_tbl").show()

+---+-------+------+-----+
| id|   name|salary|marks|
+---+-------+------+-----+
| 10|   Anil| 50000|   49|
| 11|  Vikas| 75000|   20|
| 12|  Nisha| 40000|   45|
| 13|  Nidhi| 60000|   50|
| 14|  Priya| 80000|   18|
| 15|  Mohit| 45000|   35|
| 16| Rajesh| 90000|   43|
| 17|  Raman| 55000|   39|
| 18|    Sam| 65000|   33|
|100|  Anila| 50000|   49|
|101|   Viky| 75000|   20|
|102|  Nishy| 40000|   45|
|103|   nick| 60000|   50|
|104|Pritham| 80000|   18|
+---+-------+------+-----+



In [0]:
#creating table from a data frame
#1 creating table locally which is temporary

#2 creating table globally which is temporary
    mystudentdf1.createGlobalTempView("global_mystudentdf1_table")
    # Querying the table using SQL
    spark.sql("SELECT * FROM global_temp.global_mystudentdf1_table").show()

#3 creating table permanently
emp_df.write.mode("overwrite").saveAsTable("permanent_employee_table")
   #in specific location
spark.sql("CREATE DATABASE IF NOT EXISTS company_db")
emp_df.write.mode("overwrite").saveAsTable("company_db.employee_table")

+---+------+------+-----+
| id|  name|salary|marks|
+---+------+------+-----+
| 10|  Anil| 50000|   49|
| 11| Vikas| 75000|   20|
| 12| Nisha| 40000|   45|
| 13| Nidhi| 60000|   50|
| 14| Priya| 80000|   18|
| 15| Mohit| 45000|   35|
| 16|Rajesh| 90000|   43|
| 17| Raman| 55000|   39|
| 18|   Sam| 65000|   33|
+---+------+------+-----+

